In [1]:
import numpy as np

In [3]:
nodes = {}

with open("./tspdata.txt", "r") as f:
    for line in f:
        tmp = line.strip().split()
        if len(tmp) != 3:
            continue
        nodes[int(tmp[0])] = np.array((int(tmp[1]),int(tmp[2])))

In [4]:
n = len(nodes)
d = [[0 for i in range(n + 1)] for i in range(n + 1)]
for i in  range(1,n + 1):
    for j in  range(1,n + 1):
        d[i][j] = np.math.dist(nodes[i],nodes[j])

In [5]:
def get_travel_distance(permutation):
    global d
    n = len(permutation)
    s = d[permutation[0]][permutation[-1]]
    for i in range(n-1):
        s += d[permutation[i]][permutation[i+1]]
    return s

## Self-designed neighbor operation
    

In [133]:
def neighbors(i,left = True):
    ops = []
    
    def neighbor_operator(x):
        tmp1 = list(x[:i])
        tmp2 = list(x[i:])
        if left:
            tmp2 = list(np.random.permutation(tmp2))
            tmp1.extend(tmp2)
        else:
            tmp1 = list(np.random.permutation(tmp1))
            tmp1.extend(tmp2)
        return tmp1
    
    return neighbor_operator
    

In [134]:
def generate_neighbors(n,factor = 3):
    diff = n //factor
    ops = []
    for i in range(1,diff):
        ops.append(neighbors(i))
        ops.append(neighbors(i, left = False))
    for i in range(n-1-diff, n):
        ops.append(neighbors(i))
        ops.append(neighbors(i, left = False))
    return ops

## Neighbor Operations

In [87]:
#n1
def two_opt_neighbors(x):
    x = list(x)
    n = len(x)
    cur = get_travel_distance(x)
    neighbors = []
    for i in range(0, n-3):
        for j in range(i+2, n-1):
            re = x[:i] + x[i+1:j+1][::-1] + x[j+1:]
            neighbors.append(re)
    return neighbors
#n2
def insert_one_neighbors(x):
    x = list(x)
    n = len(x)
    neighbors = []
    for i,ele in enumerate(x):
        tmp = ele
        tmp1 = x[:i]
        tmp2 = x[i+1:]
        tmp1 = tmp1 + tmp2
        for j in range(n-1):
            if i == j:
                continue
            if j == 0:
                neighbors.append([tmp] + tmp1)
            else:
                neighbors.append(tmp1[:j] + [tmp] + tmp1[j:])
    return neighbors
#n3
def swap_move_neighbors(x):
    x = list(x)
    n = len(x)
    neighbors = []
    for i in range(n-1):
        for j in range(i,n):
            if i == j:
                continue
            tmp = x[:]
            tmp[i],tmp[j] = tmp[j],tmp[i]
            neighbors.append(tmp)
    return neighbors

def change_step(x,N,k):
    f = get_travel_distance(x)
    for ele in N:
        f1 = get_travel_distance(ele)
        if f1 < f:
            return ele,k+1
    return ele,k 
    
        
        

In [136]:
def two_opt(x):
    x = list(x)
    n = len(x)
    x_ = x[:]
    cur = get_travel_distance(x)
    neighbors = []
    
    i,j = np.random.choice([i for i in range(n)],size = 2,replace = False)
    if i > j: i,j = j,i
    
    re = x_[:i+1] + x_[i+1:j+1][::-1] + x_[j+1:]
    return re

def insert_one(x):
    x = list(x)
    n = len(x)
    x_ = x[:]
    i,j = np.random.choice([i for i in range(n)],size = 2,replace = False)
    if i > j: i,j = j,i
    tmp = x_[i]
    tmp1 = x_[:i]
    tmp2 = x_[i+1:]
    tmp1 = tmp1 + tmp2
    if j == 0: re = [tmp] + tmp1
    else: re = tmp1[:j] + [tmp] + tmp1[j:]
    return re

def swap_move(x):
    x = list(x)
    n = len(x)
    x_ = x[:]
    i,j = np.random.choice([i for i in range(n)],size = 2,replace = False)
    if i > j: i,j = j,i
    re = x_[:]
    re[i],re[j] = re[j],re[i]
    return re

In [131]:
two_opt([1,2,3,4,5,6,7,8])



[1, 3, 2, 4, 5, 6, 7, 8]

In [107]:
def init_x():
    global n
    return np.random.permutation([i for i in range(1,n+1)])

In [110]:
#shake self-designed
def shake(x,k,N):
    return N[k](x)

In [111]:
def sequential_change_step(x,x1,k):
    s1 = get_travel_distance(x)
    s2 = get_travel_distance(x1)
    if s2 < s1:
        x = x1
        return x,1
    return x,k+1

In [112]:
def BVND(x,lmax,N):
    x_ = x
    while True:
        l = 0
        x1 = x_
        while l != lmax:
            x2 = N[l](x_)
            x_,l = sequential_change_step(x_,x2,l)
        #print(get_travel_distance(x1),get_travel_distance(x_))
        if get_travel_distance(x1)  <= get_travel_distance(x_):
            return x1
            

In [141]:
x = init_x()

Ns = [two_opt, insert_one, swap_move]

unchange = 10
count = 0

best_s = float('inf')
kmax = len(Ns)
#lmax = 3*kmax // 4
lmax = kmax
while count != 10000:
    k = 1
    
    while k != kmax:
        #print(k)
        x1 = shake(x,np.random.randint(kmax-1),Ns)
        x2 = BVND(x1,lmax,Ns)
        x,k = sequential_change_step(x,x2,k)
    s = get_travel_distance(x)
    print(s)
    if s < best_s:
        count = 0
        best_s = s
    else:
        count += 1

print(best_s)
        
        
    

25341.37837270813
24193.288658922094
23179.084912237908
23137.602503238177
22996.21743681804
22409.62831917413
22255.364314099512
21890.564280929677
21661.555345608514
21518.74194510315
21518.74194510315
21518.74194510315
21446.482088001783
20946.568079731514
20474.770863445287
19825.380505599973
19825.380505599973
19825.380505599973
19426.61471243613
19426.61471243613
19352.45864522175
19352.45864522175
19352.45864522175
19319.85047603644
19200.791880953275
19158.786331283136
18808.294408232414
18808.294408232414
18356.5546163182
18221.85514580178
18221.85514580178
18221.85514580178
18221.85514580178
18221.85514580178
18221.85514580178
18221.85514580178
18092.8816673309
18092.8816673309
17978.696779936003
17978.696779936003
17978.696779936003
17978.696779936003
17843.211846588543
17843.211846588543
17780.60087984774
17762.22032952867
17762.22032952867
17762.22032952867
17762.22032952867
17762.22032952867
17520.7935455262
17507.20490520281
17507.20490520281
17410.67707786467
17410.6770

11712.00819331881
11712.00819331881
11678.933175468488
11678.933175468488
11678.933175468488
11678.933175468488
11678.933175468488
11675.762827918066
11619.668607129168
11619.668607129168
11619.668607129168
11619.668607129168
11619.668607129168
11619.668607129168
11619.668607129168
11619.668607129168
11619.668607129168
11619.668607129168
11531.344290152518
11531.344290152518
11531.344290152518
11497.112838952151
11497.112838952151
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.095807165439
11459.09580716

10153.743981385074
10153.743981385074
10153.743981385074
10087.968355596926
10083.979296534359
10083.979296534359
10083.979296534359
10032.174928093198
10032.174928093198
10032.174928093198
10032.174928093198
10032.174928093198
10032.174928093198
10032.174928093198
10032.174928093198
10032.174928093198
10032.174928093198
10032.174928093198
10032.174928093198
9986.126752998118
9986.126752998118
9930.142247084252
9930.142247084252
9930.142247084252
9930.142247084252
9930.142247084252
9930.142247084252
9930.142247084252
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9915.779780382043
9895.524812373045
9895.524812373045
9895.524812373045
9895.524812373045
9895.524812373045
9895.5248

8760.00442007852
8760.00442007852
8760.00442007852
8758.261213980151
8758.261213980151
8758.261213980151
8690.254861396588
8690.254861396588
8690.254861396588
8690.254861396588
8690.254861396588
8690.254861396588
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8683.363817580323
8655.222298529758
8655.222298529758
8655.222298529758
8655.222298529758
8655.222298529758
8655.222298529758
8631.284650499278
8631.284650499278
8631.284650499278
8631.284650499278
8627.665681561977
8627.665681561977
8627.665681561977
8619.608844516037
8619.608844516037
8619.608844516037
8619.608844516037
8619.608844516037
8619.608844516037
8619.608844516037
8619.608844516037
8619.608844516037
8619.608844516037
8619.608844516037
8619.608844516037
8619.60884451

7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7768.621216322724
7763.702444398329
7763.702444398329
7763.702444398329
7763.702444398329
7763.702444398329
7763.702444398329
7763.702444398329
7743.230501045132
7743.230501045132
7743.230501045132
7743.230501045132
7743.230501045132
7743.230501045132
7743.230501045132
7743.230501045132
7743.230501045132
7743.230501045132
7729.7139936595695
7729.7139936595695
7729.7139936595695
7729.7139936595695
7729.7139936595695
7729.7139936595695
7729

...

2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813

2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813

2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813

2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813

2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813652457
2966.4095813

In [2]:
import matplotlib.pyplot as plt

In [3]:
plt.show()